In [1]:
import pandas as pd
import numpy as np
import ast
import json
from datetime import datetime


In [2]:
#aqui levanto el primer archivo
filas=[]
with open('australian_user_reviews.json', encoding='utf-8') as a:
    for linea in a.readlines():
        filas.append(ast.literal_eval(linea))

reviews = pd.json_normalize(filas)

In [3]:
a = list(reviews['reviews'])# tiene la lista de los items (lista de listas) 88310
listalarga = [item for sublist in a for item in sublist]    # esta lista es una lista con todos las reviews de dimension mayor a "a" 5153209
lista_n = [len (q) for q in a]  # esta lista tiene la cantidad n de reviews hechas por cada usuario. dimension = a 'a'88310
lista_usuarios = list(reviews['user_id'])   #dimension = a 'a' 88310


In [4]:

def repetir_elementos(lista_n, lista_u):
    #Esta funcion es utilizada para generar una serie de la misma dimension que la serie resutaltante de explotar reviews
    resultado = []
    for i in range(len(lista_n)):
        resultado.extend([lista_u[i]] * lista_n[i])
    return resultado


In [6]:
listadf = repetir_elementos(lista_n, lista_usuarios)
dfjuntado = pd.DataFrame(listalarga)
dfjuntado.insert(0,'user_id', listadf)

In [7]:
def convertir_fecha(fecha_original):
    
    try: 
        fecha_objeto = datetime.strptime(fecha_original, '%B %d, %Y')
    except:
        fecha_objeto = None

    return fecha_objeto

In [8]:

dfjuntado['posted'] = dfjuntado['posted'].str.replace('Posted ', '')
dfjuntado['posted'] = dfjuntado['posted'].str.replace('.', '')
dfjuntado['posted'] = dfjuntado['posted'].apply(convertir_fecha)    # se tienen 49186 valores con fechas correctas y 10199 con fechas incorrectas.
dfjuntado.drop(columns='helpful', inplace=True)


C:\Users\Nahuel Elias\AppData\Local\Temp\ipykernel_4168\1742476862.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfjuntado['posted'] = dfjuntado['posted'].str.replace('.', '')


In [9]:
dfjuntado['item_id'] = dfjuntado['item_id'].astype(int)
dfjuntado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      59305 non-null  object        
 1   funny        59305 non-null  object        
 2   posted       49186 non-null  datetime64[ns]
 3   last_edited  59305 non-null  object        
 4   item_id      59305 non-null  int32         
 5   recommend    59305 non-null  bool          
 6   review       59305 non-null  object        
dtypes: bool(1), datetime64[ns](1), int32(1), object(4)
memory usage: 2.5+ MB


In [28]:
dfjuntado.to_pickle("dfjuntado.pkl")


In [10]:
dfjuntado

,user_id,funny,posted,last_edited,item_id,recommend,review
0,76561197970982479,,2011-11-05,,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,2011-07-15,,22200,True,It's unique and worth a playthrough.
2,76561197970982479,,2011-04-21,,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,2014-06-24,,251610,True,I know what you think when you see this title ...
4,js41637,,2013-09-08,,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59300,76561198312638244,,NaT,,70,True,a must have classic from steam definitely wort...
59301,76561198312638244,,NaT,,362890,True,this game is a perfect remake of the original ...
59302,LydiaMorley,1 person found this review funny,NaT,,273110,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,,NaT,,730,True,:D


In [3]:
filas3=[]
with open('output_steam_games.json', encoding='utf-8') as a:
    for linea in a.readlines():
        filas3.append(json.loads(linea))
output_steams = pd.DataFrame(filas3)
steams = pd.DataFrame(filas3)
steams.dropna(subset=['genres','title', 'url', 'release_date', 'price', 'id', 'developer'], inplace= True)


In [4]:
steams['price'] = steams['price'].replace('Free to Play', '0')
steams['price'] = steams['price'].replace('Free To Play', '0')
steams['price'] = steams['price'].replace('Free', '0')
steams['price'] = pd.to_numeric(steams['price'], errors='coerce').fillna(0).astype(float)


In [7]:
steams.rename(columns={'id': 'item_id'}, inplace=True)
steams['item_id'] = steams['item_id'].astype(int)
steams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27462 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22818 non-null  object 
 1   genres        27462 non-null  object 
 2   app_name      27462 non-null  object 
 3   title         27462 non-null  object 
 4   url           27462 non-null  object 
 5   release_date  27462 non-null  object 
 6   tags          27449 non-null  object 
 7   reviews_url   27462 non-null  object 
 8   specs         27182 non-null  object 
 9   price         27462 non-null  float64
 10  early_access  27462 non-null  object 
 11  item_id       27462 non-null  int32  
 12  developer     27462 non-null  object 
dtypes: float64(1), int32(1), object(11)
memory usage: 2.8+ MB


In [8]:
steams.to_pickle("steams.pkl")

In [13]:

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to C:\Users\Nahuel
[nltk_data]     Elias\AppData\Roaming\nltk_data...


In [31]:

def analyze_sentiment(dataframe, column_name):
    # Aplica el analizador de sentimiento a cada registro de la columna y almacena los resultados en una nueva columna
    dataframe['Sentiment'] = dataframe[column_name].apply(lambda x: sia.polarity_scores(x)['compound'])
    
    # Asigna valores numéricos según la puntuación de sentimiento
    dataframe['Sentiment'] = dataframe['Sentiment'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))
    
    return dataframe

#df_analyzed = analyze_sentiment(df, 'Text')

In [32]:
df_sentiment = pd.read_pickle("dfjuntado.pkl")

In [36]:
df_sentiment = analyze_sentiment(df_sentiment, 'review')


In [46]:
df_sentiment.loc[df_sentiment['review']=='']; # con esto puede verse como la funcion analyze_sentiment retorna 1 cuando los valores de review estan vacios

In [47]:
df_sentiment.to_pickle('df_sentiment.pkl')